In [1]:
import numpy as np
import pandas as pd
import biocircuits 
import scipy.integrate

import bokeh.io
bokeh.io.output_notebook()
import panel as pn
pn.extension()

def style(p, autohide=False):
    p.title.text_font="Helvetica"
    p.title.text_font_size="16px"
    p.title.align="center"
    p.xaxis.axis_label_text_font="Helvetica"
    p.yaxis.axis_label_text_font="Helvetica"
    
    p.xaxis.axis_label_text_font_size="13px"
    p.yaxis.axis_label_text_font_size="13px"
    p.background_fill_alpha = 0
    if autohide: p.toolbar.autohide=True
    return p

Loading BokehJS ...

# <center>Homework 2.1: Designing Toggles</center>

<img src="150.2.1a.jpg" width="1500px">

I made a dashboard to view the nullcline as well as the trajectory (the trajectory is mainly for fun / not necessary to the analysis). 

Note that the initial concentrations of Ao and Bo for the trajectory plots are fixed at (Ao = 1.0, Bo = 0.0) under `# .... initializing trajectory parameters ...`

In [2]:
# .... initializing nullcline parameters .... 

betaA = 1.8
betaB = 1.8
n = 10
gamma = 1
max_xy = 5

x_A = np.linspace(0, max_xy, 400)   
x_B = np.linspace(0, max_xy, 400)

build_dict = { 'a':[ betaA * x_B**n / (1 + x_B ** n), 
                     betaB * x_A**n / (1 + x_A ** n) / gamma ],
               'b':[ betaA / (1 + x_B ** n), 
                      betaB / (1 + x_A ** n) / gamma ],
                'c':[ betaA * x_B**n / (1 + x_B ** n), 
                      betaB / (1 + x_A ** n) / gamma ], 
             }

# .... initializing nullcline plot ....

p = bokeh.plotting.figure(height=350, width=350, title="nullcline (a)", x_axis_label="A", y_axis_label="B")
q = bokeh.plotting.figure(height=350, width=350, title="nullcline (b)", x_axis_label="A", y_axis_label="B")
r = bokeh.plotting.figure(height=350, width=440, title="nullcline (c)", x_axis_label="A", y_axis_label="B")

p_cds = bokeh.models.ColumnDataSource(dict(y_A=build_dict['a'][0], x_B=x_B, y_B=build_dict['a'][1], x_A=x_A))
q_cds = bokeh.models.ColumnDataSource(dict(y_A=build_dict['b'][0], x_B=x_B, y_B=build_dict['b'][1], x_A=x_A))
r_cds = bokeh.models.ColumnDataSource(dict(y_A=build_dict['c'][0], x_B=x_B, y_B=build_dict['c'][1], x_A=x_A))

for plot, source in zip([p, q, r], [p_cds, q_cds, r_cds]):
    plot.line(source=source, x="x_A", y="y_B", color="#e97d86", line_width=3)
    plot.line(source=source, x="y_A", y="x_B", color="#2ea58e", line_width=3)

legend = bokeh.models.Legend(
            items=[ ("A", [r.line(line_color="#e97d86", line_width=3)]), 
                    ("B", [r.line(line_color="#2ea58e", line_width=3)])  ], 
            location="center"
        )
r.add_layout(legend, 'right')



# .... initializing trajectory parameters ....

time = np.linspace(0, 10, 500)
Ao, Bo = 1.0, 0.0
ABo = np.array([Ao, Bo])
β_A, β_B, γ, n = betaA, betaB, gamma, n

def derivs(AB, t, β_A, β_B, γ, n, build):
    A, B = AB
    if build in ["a", "A", "c", "C"]: A_deriv = β_A * B**n/(1+B**n) - γ*A
    else: A_deriv = β_A/(1+B**n) - γ*A
    if build in ["b", "B", "c", "C"]: B_deriv = β_B/(1+A**n) - γ*B
    else: B_deriv = β_B * A**n/(1+A**n) - γ*B
    return np.array([A_deriv, B_deriv])


# .... initializing trajectory plots .... 

s = bokeh.plotting.figure(title="trajectory (a)", height=350, width=350, 
        x_axis_label="dimensionless time", y_axis_label="dimensionless [ ]")
t = bokeh.plotting.figure(title="trajectory (b)", height=350, width=350, 
        x_axis_label="dimensionless time", y_axis_label="dimensionless [ ]")
u = bokeh.plotting.figure(title="trajectory (c)", height=350, width=440, 
        x_axis_label="dimensionless time", y_axis_label="dimensionless [ ]")

AB_s = scipy.integrate.odeint(derivs, ABo, time, args=(β_A, β_B, γ, n, 'a'))
AB_t = scipy.integrate.odeint(derivs, ABo, time, args=(β_A, β_B, γ, n, 'b'))
AB_u = scipy.integrate.odeint(derivs, ABo, time, args=(β_A, β_B, γ, n, 'c'))

A_s, B_s = AB_s.T
A_t, B_t = AB_t.T
A_u, B_u = AB_u.T

A_s /= A_s.max()
A_t /= A_t.max()
A_u /= A_u.max()

B_s /= B_s.max()
B_t /= B_t.max()
B_u /= B_u.max()

s_cds = bokeh.models.ColumnDataSource(dict(t=time, A=A_s, B=B_s))
t_cds = bokeh.models.ColumnDataSource(dict(t=time, A=A_t, B=B_t))
u_cds = bokeh.models.ColumnDataSource(dict(t=time, A=A_u, B=B_u))

for plot, source in zip([s, t, u], [s_cds, t_cds, u_cds]):
    plot.line(source=source, x='t', y='A', line_color="#e97d86", line_width=3, )
    plot.line(source=source, x='t', y='B', line_color="#2ea58e", line_width=3, )
    
legend = bokeh.models.Legend(
            items=[ ("A", [u.line(line_color="#e97d86", line_width=3)]), 
                    ("B", [u.line(line_color="#2ea58e", line_width=3)])
                  ], location="center")
u.add_layout(legend, 'right')

In [ ]:
# .... building widgets ....

betaA_slider = bokeh.models.Slider(title="βA",   start=0.1, end=5.0, step=0.1, value=1.8, width=250)
betaB_slider = bokeh.models.Slider(title="βB",   start=0.1, end=5.0, step=0.1, value=1.8, width=250)
gamma_slider = bokeh.models.Slider(title="γ",    start=0.1, end=5.0, step=0.1, value=1.0, width=250)
n_slider     = bokeh.models.Slider(title="n",    start=1.0, end=25., step=0.1, value=10., width=250)
max_slider   = bokeh.models.Slider(title="max",  start=2.0, end=10., step=0.1, value=5.0, width=250)
t_max_slider = bokeh.models.Slider(title="time", start=10., end=20., step=2.0, value=10., width=250)

def callback(attr, old, new):
    x_A = np.linspace(0, max_slider.value, 400)   
    x_B = np.linspace(0, max_slider.value, 400)
    p_cds.data["y_A"] = betaA_slider.value * x_B**n_slider.value / (1 + x_B ** n_slider.value)
    p_cds.data["y_B"] = betaB_slider.value * x_A**n_slider.value / (1 + x_A ** n_slider.value) / gamma_slider.value 
    
    q_cds.data["y_A"] = betaA_slider.value / (1 + x_B ** n_slider.value) 
    q_cds.data["y_B"] = betaB_slider.value / (1 + x_A ** n_slider.value) / gamma_slider.value 
    
    r_cds.data["y_A"] = betaA_slider.value * x_B**n_slider.value / (1 + x_B ** n_slider.value)
    r_cds.data["y_B"] = betaB_slider.value / (1 + x_A ** n_slider.value) / gamma_slider.value

    β_A, β_B, γ, n = betaA_slider.value, betaB_slider.value, gamma_slider.value, n_slider.value
    
    time = np.linspace(0, t_max_slider.value, 500)
    AB_s = scipy.integrate.odeint(derivs, ABo, time, args=(β_A, β_B, γ, n, 'a'))
    AB_t = scipy.integrate.odeint(derivs, ABo, time, args=(β_A, β_B, γ, n, 'b'))
    AB_u = scipy.integrate.odeint(derivs, ABo, time, args=(β_A, β_B, γ, n, 'c'))

    A_s, B_s = AB_s.T
    A_t, B_t = AB_t.T
    A_u, B_u = AB_u.T

    A_s /= A_s.max()
    A_t /= A_t.max()
    A_u /= A_u.max()

    B_s /= B_s.max()
    B_t /= B_t.max()
    B_u /= B_u.max()

    s_cds.data["A"] = A_s
    s_cds.data["B"] = B_s  
    t_cds.data["A"] = A_t
    t_cds.data["B"] = B_t    
    u_cds.data["A"] = A_u
    u_cds.data["B"] = B_u    
    
    
# .... linking widgets ....

betaA_slider.on_change("value", callback)
betaB_slider.on_change("value", callback)
gamma_slider.on_change("value", callback)
n_slider.on_change("value", callback)
max_slider.on_change("value", callback)
t_max_slider.on_change("value", callback)

# .... making layouts ....

lay_widgets = bokeh.layouts.Row( bokeh.layouts.Column(betaA_slider, betaB_slider), 
                                 bokeh.layouts.Column(gamma_slider, n_slider), align="center")
lay_time = bokeh.layouts.Row(max_slider, align="center")
lay_nullcline = bokeh.layouts.Row(style(p), style(q), style(r))
lay_trajectory = bokeh.layouts.Row(style(s), style(t), style(u))

lay_time_max = bokeh.layouts.Row(t_max_slider, align="center")
layout = bokeh.layouts.Column(lay_nullcline, lay_widgets, lay_time, lay_trajectory, lay_time_max)

# # .... exporting HTML purposes .... 
# import os
# import sys
# sys.stderr = open(os.devnull, "w")  # silence stderr
# bokeh.io.show(layout)

# .... serving dashboard .... 
def app(doc):
    doc.add_root(layout)
bokeh.io.show(layout)

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html



# observations:
- Slide β_A around to see the genetic toggle / steady states in action (look at trajectory B plot)!
- Note that in nullcline (b) A = β_A, and B = β_B, as noted above in the pdf.
- We seen increasing n increases oscillations in trajectory c. 
- In, nullcline A that middle steady state was unexpeected, but it makes sense since when A is high and B is low, A will keep getting degraded, but it is simultaneously activating B and once there is enough B, the concentration in A recovers

I am, as always, afraid my dashboard didn't work, so here's a screenshot. In the case it did work, apologies for the redundancy!
<img src="dash0.png">

<img src="150.2.1b.jpg" width="850px">

<div class="alert alert-block alert-info">
Excellent analysis! 45/45 <br>
Graded by AR

</div>